In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

In [ ]:
# silence warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [ ]:
%matplotlib inline

%reload_ext autoreload
%autoreload 2

import keras

from keras_retinanet.models import retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

import matplotlib.pyplot as plt
import cv2
import numpy as np
import time
import json

In [ ]:
! ../../../keras-retinanet/keras_retinanet/bin/train.py --weights ../../../resnet50_coco_best_v2.1.0.h5 \
--batch-size 2 --steps 500 --epochs 20 \
--snapshot-path ../../Data/elephant-seals/snapshots-with-other \
--random-transform \
--config ../../Data/elephant-seals/config-with-other.ini \
csv ../../Data/elephant-seals/train/annotations_final.csv ../../Data/elephant-seals/classes-with-other.csv \
--val-annotations ../../Data/elephant-seals/valid/annotations_final.csv

In [ ]:
# convert the model weights to a usable format
! ../../../keras-retinanet/keras_retinanet/bin/convert_model.py --config ../Data/image-level-split/elephant-config-image-level.ini  ../../Data/elephant-seals/snapshots-with-other/resnet50_csv_18.h5 ../Data/image-level-split/model_18_epochs.h5
# run the model on test set
! ../../../keras-retinanet/keras_retinanet/bin/evaluate.py csv ../Data/image-level-split/test/annotations_final.csv ../Data/elephant_classes.csv ../Data/image-level-split/model_18_epochs.h5

In [ ]:
def get_session():
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.compat.v1.Session(config=config)

tf.compat.v1.keras.backend.set_session(get_session())

In [ ]:
model_path = '../../Data/elephant-seals/models/elephant_seal_model_with_other.h5'

model = models.load_model(model_path, backbone_name='resnet50')

labels_to_names = {0: 'bull', 1: 'cow', 2: 'pup', 3: 'other'}

In [ ]:
image_dir = "../../Data/elephant-seals/test/"

image_list = []
for root, dirs, files in os.walk(image_dir):
    for filename in files:
        if filename.lower().endswith(('.jpg')):
            image_list.append(image_dir + filename)

print(len(image_list))

In [ ]:
min_score = 0.5

detections = {}

total_time = 0

for image_path in image_list:
    
    image = read_image_bgr(image_path)
          
    image = preprocess_image(image)
    image, scale = resize_image(image)
    
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    total_time += time.time() - start
    
    boxes /= scale  
        
    if any(score >= min_score for score in scores[0]):
        detections[image_path] = []
        
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score < min_score:
            break
        
        b = box.astype(int)    
        detections[image_path].append({"box" : b, "label" : label, "score" : score})
        
            
print("Finished, time per image:", total_time/len(image_list))

In [ ]:
# output model detections
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)
        
with open('../Data/image-level-split/test_detections.json', 'w') as fp:
    json.dump(detections, fp, cls=MyEncoder)

In [ ]:
random_image_list = [
    '../Data/image-level-split/test/5MSL3749-174_png.rf.1ade9f0908936fd853dc2ae9a0f00233.jpg',
     '../Data/image-level-split/test/5MSL3764-21_png.rf.d67ea7373c5b15df832b5e11d4d59192.jpg',
     '../Data/image-level-split/test/5MSL3819-65_png.rf.7cf3c5254e14e6b5cf5905c04ef3f848.jpg',
     '../Data/image-level-split/test/5MSL0109-125_png.rf.0999b3e2f7414ce671eb6ece5f074af1.jpg',
     '../Data/image-level-split/test/5MSL0109-4_png.rf.4c177bee4ff3cdfb044448de4af8f62c.jpg',
     '../Data/image-level-split/test/5MSL0087-138_png.rf.6843d941557d06ab666d79fe30071175.jpg',
     '../Data/image-level-split/test/5MSL4416-99_png.rf.1e915260973f7fd233ef130506a02836.jpg',
     '../Data/image-level-split/test/4MSL0120-163_png.rf.fec5aded76dfdbf287db0a33cffecb65.jpg',
     '../Data/image-level-split/test/5MSL3749-24_png.rf.c0d599cb5e1a10d9bb787cbdbef5e201.jpg',
     '../Data/image-level-split/test/5MSL1479-6_png.rf.c9a1b45ca0cc3abd059facca39256aff.jpg',
     '../Data/image-level-split/test/5MSL3749-146_png.rf.adb2315a6cc0e239980419d05ae2fcab.jpg',
     '../Data/image-level-split/test/4MSL0119-104_png.rf.0f73c19f543a90b15c12dc72bf972446.jpg',
     '../Data/image-level-split/test/5MSL0087-120_png.rf.ea046df0b3121de73453abcfc2f6fcb6.jpg',
     '../Data/image-level-split/test/5MSL0109-84_png.rf.e1d78f59110465473cd83883edd58ab0.jpg',
     '../Data/image-level-split/test/5MSL3436-6_png.rf.4ea434812bb5b4e80c37b85c5103168d.jpg',
     '../Data/image-level-split/test/5MSL0095-98_png.rf.d0a2d706df7b447719994fd7de1f7d36.jpg',
     '../Data/image-level-split/test/5MSL0087-22_png.rf.b500d2b83eb6a2b9d76fcd6cbe9c55af.jpg',
     '../Data/image-level-split/test/5MSL0109-95_png.rf.73ae554e192ba0a1cb271ec123f8330f.jpg',
     '../Data/image-level-split/test/5MSL3436-15_png.rf.10de22d575068c5c4d52c255e5c56cfa.jpg',
     '../Data/image-level-split/test/4MSL0120-169_png.rf.374080801c483f31562d9e265eb66652.jpg',
     '../Data/image-level-split/test/MA263076-124_png.rf.42e9d6beed2e154f9051414a40aae592.jpg',
     '../Data/image-level-split/test/5MSL0089-142_png.rf.00c837be2517e38115a9bcd28e1acb21.jpg',
     '../Data/image-level-split/test/5MSL3764-132_png.rf.eaaa0fa497455e8c42fb6698f4f1f125.jpg',
     '../Data/image-level-split/test/5MSL3436-115_png.rf.d2ebe4ef97473624b43c17e3373c338d.jpg',
     '../Data/image-level-split/test/5MSL3764-18_png.rf.7dd55fc701e3f0ec09ef893668bbb45d.jpg',
     '../Data/image-level-split/test/5MSL3764-134_png.rf.294fa1f2ec3227e20995494de77ac3f4.jpg',
     '../Data/image-level-split/test/5MSL3764-76_png.rf.2bfc3f67fdfefc7715222d7aad08ae9c.jpg',
     '../Data/image-level-split/test/4MSL0120-140_png.rf.03e7f0b7cf647b4b8793a1fbaa15844e.jpg',
     '../Data/image-level-split/test/5MSL3819-55_png.rf.a7ec0f8019e2d7ee1d781867ea416e53.jpg',
     '../Data/image-level-split/test/5MSL0087-112_png.rf.d74dfca1a15442986c922680263769f5.jpg',
     '../Data/image-level-split/test/5MSL0019-47_png.rf.abd556ad1262d88c05f035864ab95be2.jpg',
     '../Data/image-level-split/test/MA262988-171_png.rf.6d746f67067125adb31283c14052f12c.jpg',
     '../Data/image-level-split/test/4MSL0120-175_png.rf.93c8f9d5153f911143651f418f06eacb.jpg',
     '../Data/image-level-split/test/4MSL0119-148_png.rf.e00ecbfdbf3158afb96f78f40ceecfcd.jpg',
     '../Data/image-level-split/test/4MSL0119-113_png.rf.0c51d0505db734bdeae1913e43760ebf.jpg',
     '../Data/image-level-split/test/4MSL0119-67_png.rf.758a0d45240eaa61ad24cf86298f3332.jpg',
     '../Data/image-level-split/test/5MSL3819-48_png.rf.b37d718edf1faf1f279c760d23685211.jpg',
     '../Data/image-level-split/test/5MSL3819-168_png.rf.c791f19bc213d15a6cb4a1c8839261c0.jpg',
     '../Data/image-level-split/test/5MSL0049-23_png.rf.aa4142b069ab92c6eafd7f01633095d5.jpg',
     '../Data/image-level-split/test/5MSL4383-68_png.rf.5360242ffe1ca1a0da6d0062d96169b0.jpg']

In [ ]:
# visualize adjusted output
visualize = True

for image_path in random_image_list: 
    
    image = read_image_bgr(image_path)
    print(image_path)
    
    if visualize:
        draw = image.copy()
        draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
        
    image = preprocess_image(image)
    image, scale = resize_image(image)
    
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    total_time += time.time() - start 
    
    boxes /= scale
    if any(score >= min_score for score in scores [0]):
        detections[image_path] = []
    
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score < min_score:
            break
        
        b = box.astype(int)
        detections[image_path].append({"box" : b, "label" : label, "score" : score})
        
        # don't visualize the annotations along the edge of the tile
        if b[0] == 0 or b[1] == 0 or b[2] == 499 or b[3] == 499:
            continue
        
        if visualize: 
            color = label_color(label)
            
            draw_box(draw, b, color=color)
            
            # # uncomment the following to see labels and confidence scores
            # caption = "{} {:.3f}".format(labels_to_names[label], score)
            # draw_caption(draw, b, caption)
            
    if any(score >= min_score for score in scores[0]):
        if visualize:
            plt.figure(figsize=(15, 15))
            plt.axis('off')
            plt.imshow(draw)
            plt.show()
            
print("Finished, time per image:", total_time/len(random_image_list))